# Messaging


### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored
import asyncio


from libs.messaging_service import MessagingService

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://demo-participant-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Optional - Configure a Default Mediator

We have hosted an ACA agent in open mediation mode. Accept this connection and request mediation from it and it should grant your request. If you set this as your default mediation all future invites will have serviceEndpoints pointing to this agent which will then forward messages on to your endpoint.

In [5]:
mediator_invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '9f0ce195-3ad4-4b3e-ae87-8b89703df0bc', 'label': 'HGF_ACA_Playground_Mediator', 'recipientKeys': ['GdXBx75Mj2RikEdJfvPTpDYTnR8xLFrX6tDqDiwjDUcy'], 'serviceEndpoint': 'http://139.162.224.50:6666'}

In [12]:
invitation_response = await agent_controller.connections.receive_invitation(mediator_invite)

In [13]:
connection_id = invitation_response["connection_id"]
### check state of connection
connection = await agent_controller.connections.get_connection(connection_id)

# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection between the mediator agent and Alice's subwallet agent")    
else:
    ## request mediation
    mediation_req = await agent_controller.mediation.request_mediation(connection_id)
    

In [15]:
### To have a look at the mediation records and we should see our mediation in there
records_response = await agent_controller.mediation.get_mediation_records()
print(f"State : {records_response[1]['state']}")

State : request


In [ ]:
default_mediation_res = await agent_controller.mediation.set_default_mediator(response[0]['mediation_id'])
print(default_mediation_res)

## Instatiate Messaging Service

In [4]:
messaging_service = MessagingService(agent_controller)

## Add Contact

### Either from Invitation Shared

In [5]:
invitation = {"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "b352adae-24fe-44d0-a6dc-43c853242d17", "serviceEndpoint": "https://3a8742dc608e.ngrok.io", "label": "Hyperledger Global Forum", "recipientKeys": ["JE9d6umnBNqrdftsa9nLG2ExkzjdyVPrG9ghGMZ7UNxP"]}


In [6]:
new_contact_id = messaging_service.accept_contact_invitation(invitation, "HLGF")

Contact with ID 9f096450-fa37-499b-9016-72360495076b and alias HLGF added
Contact with ID: 9f096450-fa37-499b-9016-72360495076b and alias HLGF is now active.


### Or by Creating and Sharing an Invite

In [13]:
alias = "HLGF"
response = messaging_service.new_contact_invite(alias)

# If you want to display as Q.R for example
invitation = response["invitation"]

new_contact_id = response["connection_id"]

Contact with ID 7069051b-0ea6-427b-8fea-3421f41e1035 and alias HLGF added
----------------------------------------------------------------------------------------------------

Share this invite object with another entity you wish to add as a contact under alias HLGF

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "520fda73-c3df-4be4-a34a-5dd79f3c289b", "serviceEndpoint": "https://b7809cc28248.ngrok.io", "label": "Demo Participant", "recipientKeys": ["DyZpe9vf8Fd6DYBkMXwFoxGcqr3kAvfTBNLau17DgdN8"]}


----------------------------------------------------------------------------------------------------


## Send Message to Contact by Connection ID

In [9]:
some_arbitrary_sized_message = "Hey how are you, want to grab a coffee?"
messaging_service.send_message(new_contact_id, some_arbitrary_sized_message)

## Display Inbox for Contact

In [10]:
messaging_service.display_inbox_for_contact(new_contact_id)

--------------------------------------------------
Inbox for Contact HLGF
--------------------------------------------------
Connection ID : 9f096450-fa37-499b-9016-72360495076b 
Agent Label : Hyperledger Global Forum 
Total Messages : 2
--------------------------------------------------
Received                                      Sent
--------------------------------------------------
Thu Jun  3 14:57:57 2021
                         Want to grab a coffee? 
--------------------------------------------------
Thu Jun  3 14:58:14 2021
                         Hey how are you, want to grab 
                         a coffee? 
--------------------------------------------------


## Get Contacts

In [14]:
messaging_service.view_inbox(show_inactive=True)

--------------------------------------------------
2 Contacts
--------------------------------------------------
Contact HLGF
ID: 9f096450-fa37-499b-9016-72360495076b 
Label: Hyperledger Global Forum
Status : ACTIVE 
Messages 2 
--------------------------------------------------
Contact HLGF
ID: 7069051b-0ea6-427b-8fea-3421f41e1035 
Label: None
Status : INACTIVE 
Messages 0 
--------------------------------------------------



In [12]:
response = messaging_service.get_contacts()
for contact in response:
    print(f"{contact.alias} : {contact.connection_id}")
    print(contact.is_active.done())
    print(contact.__dict__)

HLGF : 9f096450-fa37-499b-9016-72360495076b
True
{'connection_id': '9f096450-fa37-499b-9016-72360495076b', 'alias': 'HLGF', 'agent_label': 'Hyperledger Global Forum', 'inbox': [<libs.messaging_service.message.Message object at 0x7fa77348b4d0>, <libs.messaging_service.message.Message object at 0x7fa773490990>], 'is_active': <Future finished result=True>}


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()